In [ ]:
#导入相关包
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn2pmml import PMMLPipeline

#读入数据
iris = load_iris()
X_train=pd.DataFrame(iris.data,columns=['sepal_length','sepal_width','petal_length', 'petal_width'])
y_train=pd.DataFrame(iris.target,columns=['series'])

#训练模型pipeline
clf = tree.DecisionTreeClassifier(max_depth=2) #定义分类器
pipeline = PMMLPipeline([("classifier", clf)]) #定义pipeline
pipeline.fit(X_train, y_train) #此处使用带columns变量名称的dataframe进行模型训练

#模型导出为PMML
from sklearn2pmml import sklearn2pmml
sklearn2pmml(pipeline, "DecisionTree_Iris_sklearn2pmml.pmml", with_repr = True) #生成PMML时带变量名

In [ ]:
# 下面代码使用Flask包实现在服务器部署模型
# （1）首先将下面代码保存在服务器端，命名为main.py，然后在服务器端执行命令行：python main.py
#导入相关包和模块
import numpy as np
import pandas as pd
from pypmml import Model
from flask import Flask
from flask import request
from flask import jsonify


#导入模型
model = Model.fromFile("DecisionTree_Iris_sklearn2pmml.pmml")
 
app = Flask(__name__)
 
@app.route('/',methods=['POST','GET'])
def scoring():
    text=request.args.get('inputdata')
    if text:
        temp =  [float(x) for x in text.split(',')]
        temp = pd.DataFrame(data=np.array(temp).reshape((1, -1)),columns=["sepal_length", "sepal_width", "petal_length", "petal_width"])
        ouputdata = model.predict(temp)	#outputdata是DataFrame格式
        return jsonify(dict(ouputdata.iloc[0])) #进行json化
        
if __name__ == '__main__':
    app.config['JSON_AS_ASCII'] = False
    app.run(host='127.0.0.1',port=5003)  # 127.0.0.1 #指的是本地ip

In [1]:
# （2）客户端执行如下代码，在服务器模式下执行时将127.0.0.1替换为服务器IP地址
import requests

base = 'http://127.0.0.1:5003/?inputdata=5.1,3.5,1.4,2'
response = requests.get(base)
print(response.text)
answer = response.json()
print('预测结果',answer)

{"probability(0)":1.0,"probability(1)":0.0,"probability(2)":0.0}

预测结果 {'probability(0)': 1.0, 'probability(1)': 0.0, 'probability(2)': 0.0}
